In [18]:
import pandas as pd
import time
import numpy as np
from datetime import datetime

In [32]:
df = pd.read_csv('/Users/advait_t/Desktop/Jio/Stock_Prediction/Stock_Prediction/data/final/error_df1ITC.NS.csv')
df.tail()

,Date,Actual_Close,Predicted_Close,Predicted_Close_Minimum,Predicted_Close_Maximum,Percent_Change_from_Close,Actual_Up_Down,Predicted_Up_Down,Company
45,2022-09-07,327.899994,336.965541,327.291411,346.212697,-2.764729,Down,Down,ITC.NS
46,2022-09-08,NaN,334.426330,324.396771,344.506276,NaN,Down,Down,ITC.NS
47,2022-09-09,330.500000,337.477638,327.943744,347.532571,-2.111237,Down,Up,ITC.NS
48,2022-09-12,330.700012,340.386257,330.434030,351.211982,-2.929013,Down,Down,ITC.NS
49,2022-09-13,NaN,343.090985,332.906912,353.002745,NaN,Down,Up,ITC.NS


In [31]:
# # #! Check for null values in actual close and get its date
# null_values = df[df['Actual_Close'].isnull()]
# # add %H:%M:%S to get time as well
# null_values['Date'] = pd.to_datetime(null_values['Date'], format = '%Y-%m-%d %H:%M:%S')
# # convert to string
# null_values['Date'] = null_values['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')
# null_values = null_values['Date'].to_list()

In [33]:
# Fetch stock price when there is a null value in actual price column
def fetch_stock_price(company, date):
    now = datetime.now()

    past = datetime.strptime(str(date), '%Y-%m-%d %H:%M:%S')
    past = past.replace(hour = now.hour, minute = now.minute, second = now.second, microsecond = now.second)
    past = int(time.mktime(past.timetuple()))
    
    interval = '1d'

    # defining the query to get historical stock data
    query_string = f'https://query1.finance.yahoo.com/v7/finance/download/{company}?period1={past}&period2={past}&interval={interval}&events=history&includeAdjustedClose=true'
    
    try:
        company_stock_price = pd.read_csv(query_string)
        company_stock_price = company_stock_price[['Date', 'Close']]
        return company_stock_price['Close'][0]
    except:
        return None

def update_actual_close(df, company):
    #! Check for null values in actual close and get its date
    null_values = df[df['Actual_Close'].isnull()]
    # add %H:%M:%S to get time as well
    null_values['Date'] = pd.to_datetime(null_values['Date'], format = '%Y-%m-%d %H:%M:%S')
    # convert to string
    null_values['Date'] = null_values['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')
    null_values = null_values['Date'].to_list()

    for date in null_values:
        stock_price = fetch_stock_price(company, date)
        # remove time from date
        date = date.split(' ')[0]
        # append to dataframe
        df.loc[df['Date'] == date, 'Actual_Close'] = stock_price
        # calculate percent change from close for the date in null_values
        df.loc[df['Date'] == date, 'Percent_Change_from_Close'] = (df.loc[df['Date'] == date, 'Predicted_Close'] - df.loc[df['Date'] == date, 'Actual_Close'])/df.loc[df['Date'] == date, 'Actual_Close']

    df['Actual_Up_Down'] = np.where((df['Actual_Close'] > df['Actual_Close'].shift(-1)), 'Up', 'Down')
    df['Predicted_Up_Down'] = np.where((df['Predicted_Close'] > df['Actual_Close'].shift(-1)), 'Up', 'Down')

    return df

    

In [34]:
update_actual_close(df, 'ITC.NS')

/Users/advait_t/.pyenv/versions/3.7.3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/advait_t/.pyenv/versions/3.7.3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Date,Actual_Close,Predicted_Close,Predicted_Close_Minimum,Predicted_Close_Maximum,Percent_Change_from_Close,Actual_Up_Down,Predicted_Up_Down,Company
0,2022-07-04,291.899994,NaN,NaN,NaN,NaN,Up,Down,ITC.NS
1,2022-07-05,286.899994,280.266958,271.719353,288.807245,2.311968,Down,Down,ITC.NS
2,2022-07-06,289.299988,282.532617,274.502030,290.236457,2.339223,Down,Down,ITC.NS
3,2022-07-07,291.950012,283.349795,274.079533,291.798510,2.945784,Down,Down,ITC.NS
4,2022-07-08,293.049988,283.643808,275.980434,291.614709,3.209753,Down,Down,ITC.NS
5,2022-07-11,295.000000,284.351308,276.362693,292.621246,3.609726,Up,Down,ITC.NS
6,2022-07-12,292.700012,284.535902,276.465611,292.791929,2.789241,Down,Down,ITC.NS
7,2022-07-13,294.049988,286.311465,277.823273,294.554911,2.631703,Up,Down,ITC.NS
8,2022-07-14,290.850006,288.909939,280.668226,297.403170,0.667034,Down,Down,ITC.NS
9,2022-07-15,293.549988,289.075030,280.969307,297.056535,1.524428,Down,Down,ITC.NS


In [29]:
# # use the function to fetch stock price and append to the dataframe for that date
# for date in null_values:
#     stock_price = fetch_stock_price('ITC.NS', date)
#     # remove time from date
#     date = date.split(' ')[0]
#     # append to dataframe
#     df.loc[df['Date'] == date, 'Actual_Close'] = stock_price
#     # calculate percent change from close for the date in null_values
#     df.loc[df['Date'] == date, 'Percent_Change_from_Close'] = (df.loc[df['Date'] == date, 'Predicted_Close'] - df.loc[df['Date'] == date, 'Actual_Close'])/df.loc[df['Date'] == date, 'Actual_Close']

# df['Actual_Up_Down'] = np.where((df['Actual_Close'] > df['Actual_Close'].shift(-1)), 'Up', 'Down')
# df['Predicted_Up_Down'] = np.where((df['Predicted_Close'] > df['Actual_Close'].shift(-1)), 'Up', 'Down')



In [30]:
# df.tail(20)

In [35]:
error_df_path = '/Users/advait_t/Desktop/Jio/Stock_Prediction/Stock_Prediction/data/final/error_df1'
error_df_path + 'itc' + '.csv'

'/Users/advait_t/Desktop/Jio/Stock_Prediction/Stock_Prediction/data/final/error_df1itc.csv'